# **Convolutional Neural Network: Leaves Classification**
**Artificial  Neural  Networks  and  Deep  Learning  -  a.y.  2021/2022**

*     <u>Fabio Tresoldi</u>
> M.Sc. Computer Science and EngineeringPolitecnico di Milano - Milan, Italy
>
> E-mail: fabio1.tresoldi@mail.polimi.it
>
> Student ID : 10607540
>
> Codalab Nickname: "fabioow"
>
> Codalab Group: "artificial_comrades"

*     <u>Mirko  Usuelli</u>
> M.Sc. Computer Science and EngineeringPolitecnico di Milano - Milan, Italy
>
>E-mail: mirko.usuelli@mail.polimi.it
>
>Student ID : 10570238
>
>Codalab Nickname: "mirko"
>
>Codalab Group: "artificial_comrades"


## Environment settings

### Libraries

In [ ]:
import os
import random
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

tfk = tf.keras
tfkl = tf.keras.layers

print(tf.__version__)

### Random seed

In [ ]:
# Random seed for reproducibility
SEED = 42

random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.compat.v1.set_random_seed(SEED)

## EfficientNet B0 Fine Tuning

### Metadata

In [ ]:
# Labels
LABELS = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

In [ ]:
# Model hyperparameters
INPUT_SHAPE = (256, 256, 3)
IMAGE_SIZE = (INPUT_SHAPE[0], INPUT_SHAPE[1])
EPOCHS = 100
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
NUM_CLASSES = len(LABELS)
MODEL_NAME = "cnn"

### Data Loader
This notebook assumes to have a dataset already divided into the usual subsets (i.e. training, validation, testing).
This is because we decided to split the dataset *offline* (i.e. by using a python script in our local machines, see `Data_Splitter.ipynb`).

This solution has been adopted so we could use `ImageDataGenerator` for the pre-processing and the augmentation of the images as seen during the course. Moreover, since the dataset has unbalanced classes, a stratified split has been applied.

In [ ]:
# Paths
ROOT_PATH = 'leaf_dataset_splitted/' # The name of the directory cotaining the splitted dataset, it should be in the same directory of this notebook
TRAINING_DIR = os.path.join(ROOT_PATH, 'training')
VALIDATION_DIR = os.path.join(ROOT_PATH, 'validation')
TESTING_DIR = os.path.join(ROOT_PATH, 'testing')

### Data Pre-Processing and Augmentation
*     Pre-Processing with `tensorflow.keras.applications.efficientnet.preprocess_input`
*     Augmentation with `ImageDataGenerator`





#### Image Generators

##### Training set

In [ ]:
# Constructor
train_data_gen = ImageDataGenerator(
    # Data Augmentation
    rotation_range=30,
    height_shift_range=50,
    width_shift_range=50,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.7,1.3],
    fill_mode='reflect',
    preprocessing_function=preprocess_input
)

# Generator
train_gen = train_data_gen.flow_from_directory(
    directory=TRAINING_DIR,
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

##### Validation set

In [ ]:
# Constructor
valid_data_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# Generator
valid_gen = valid_data_gen.flow_from_directory(
    directory=VALIDATION_DIR,
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

##### Testing set

In [ ]:
# Constructor
test_data_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# Generator
test_gen = test_data_gen.flow_from_directory(
    directory=TESTING_DIR,
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False,
    seed=SEED
)

### Model Design

#### Utilities

In [ ]:
# Utility function to create folders and callbacks for training
def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('models')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  exp_dir = os.path.join(exps_dir, model_name)
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=False) # True to save only the best epoch 
  callbacks.append(ckpt_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

#### Fine Tuning Model



##### Download EfficientNet B0

In [ ]:
# Download and plot the EfficientNet B0 model
supernet = tfk.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=INPUT_SHAPE
)

# Plot the model
supernet.summary()
tfk.utils.plot_model(supernet)

##### Rebuild the Top (i.e. Classifier)

In [ ]:
# Use the supernet as feature extractor
supernet.trainable = False

# Rebuild the classifier
inputs = tfk.Input(shape=INPUT_SHAPE)
x = supernet(inputs)
x = tfkl.GlobalAveragePooling2D()(x)
x = tfkl.Dense(256, kernel_initializer = tfk.initializers.GlorotUniform(SEED))(x)
x = tfkl.BatchNormalization()(x)
x = tfkl.ReLU()(x)
outputs = tfkl.Dense(
    NUM_CLASSES, 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(SEED))(x)

# Connect input and output through the Model class
model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=LEARNING_RATE), metrics=['accuracy'])
model.summary()

##### Freeze the first 162 layers of EfficientNet

In [ ]:
# Set all EfficientNet layers to True
model.get_layer('efficientnetb0').trainable = True
for i, layer in enumerate(model.get_layer('efficientnetb0').layers):
   print(i, layer.name, layer.trainable)

In [ ]:
# Freeze the first 162 layers
for i, layer in enumerate(model.get_layer('efficientnetb0').layers[:162]):
  layer.trainable=False

print()
for i, layer in enumerate(model.get_layer('efficientnetb0').layers):
   print(i, layer.name, layer.trainable)
model.summary()

In [ ]:
# Compile the model
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=LEARNING_RATE), metrics=['accuracy'])
model.summary()

### Training and Validation (Fine Tuning)

In [ ]:
# Create folders and callbacks
callbacks = create_folders_and_callbacks(model_name=MODEL_NAME)

# Train the model
history = model.fit(
    x = train_gen,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    validation_data = valid_gen,
    callbacks=[callbacks]
).history

In [ ]:
# Plot the training history
plt.figure(figsize=(10,5))
plt.plot(history['loss'], alpha=.3, color='r', linestyle='--', linewidth=3)
plt.plot(history['val_loss'], label='EfficientNet B0', alpha=.8, color='r', linewidth=3)
plt.legend(loc='upper right', prop={'size': 18})
plt.title('Categorical Crossentropy', fontsize=20)
plt.grid(alpha=.3)

plt.figure(figsize=(10,5))
plt.plot(history['accuracy'], alpha=.3, color='r', linestyle='--', linewidth=3)
plt.plot(history['val_accuracy'], label='EfficientNet B0', alpha=.8, color='r', linewidth=3)
plt.legend(loc='upper right', prop={'size': 18})
plt.title('Accuracy', fontsize=20)
plt.grid(alpha=.3)

plt.show()

### Testing

In [ ]:
test_steps_per_epoch = np.math.ceil(test_gen.samples / test_gen.batch_size)

# Evaluate on test
predictions = model.predict(test_gen, steps=test_steps_per_epoch)

In [ ]:
# Get most likely classes
predicted_classes = np.argmax(predictions, axis=-1)

In [ ]:
# Get true classes
true_classes = test_gen.classes
class_labels = list(test_gen.class_indices.keys())

In [ ]:
# Compute the confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Compute the classification metrics
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='macro')
recall = recall_score(true_classes, predicted_classes, average='macro')
f1 = f1_score(true_classes, predicted_classes, average='macro')
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

# Plot the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm.T, xticklabels=list(class_labels), yticklabels=class_labels)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

### Save the Model

In [ ]:
# Save best epoch model
model.save("models/cnn_best")